# HOG-SVM Klasifikasi Citra Alien dan Predator

In [1]:
import numpy as np 
import matplotlib.pyplot as plt
from sklearn import datasets
from skimage.feature import hog
from skimage import data, exposure
import cv2
import os
import sys

### Class SVM  

In [2]:
class SVM:

    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=1000):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.w = None
        self.b = None


    def fit(self, X, y):
        n_samples, n_features = X.shape
        
        y_ = np.where(y <= 0, -1, 1)
        
        self.w = np.zeros(n_features)
        self.b = 0

        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                condition = y_[idx] * (np.dot(x_i, self.w) - self.b) >= 1
                if condition:
                    self.w -= self.lr * (2 * self.lambda_param * self.w)
                else:
                    self.w -= self.lr * (2 * self.lambda_param * self.w - np.dot(x_i, y_[idx]))
                    self.b -= self.lr * y_[idx]


    def predict(self, X):
        approx = np.dot(X, self.w) - self.b
        return np.sign(approx)

### Fungsi untuk mengambil citra dari direktori  

In [3]:
def load_images(folder):
    images = np.array([])
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.resize(img,(200,200))
        fd, hog_image = hog(img, orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1), visualize=True, multichannel=True)
        hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))
        if img is not None:
            hog_image_rescaled = hog_image_rescaled.flatten()
            images= np.append(images,hog_image_rescaled)
    return images

In [4]:
imgdata = load_images('./data/train/data')

### Panjang Fitur HOG dari seluruh data latih dengan dimensi 200x200

In [5]:
len(imgdata)

27760000

In [6]:
labels = np.array([])
for i in range(347):
    labels = np.append(labels,1)

In [7]:
for i in range(347):
    labels = np.append(labels,-1)

In [9]:
imgdata = imgdata.reshape(694,40000)

In [10]:
len(imgdata)

694

#### Proses pelatihan SVM SMO dengan masukkan parameter HOG dan Label Data berupa Alien = 1 dan Predator = -1

In [11]:
clf = SVM()
clf.fit(imgdata, labels)

### Class SVM  

In [12]:
def load_predict(folder):
    predictions = np.array([])
    w= np.array([])
    b= np.array([])
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.resize(img,(200,200))
        fd, hog_image = hog(img, orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1), visualize=True, multichannel=True)
        hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))
        if img is not None:
            predicted = hog_image_rescaled.flatten()
            prediction = clf.predict(predicted)
#             print(clf.w, clf.b, prediction)
#             predictions = np.array(prediction)
            w = np.append(w,clf.w)
            b = np.append(b,clf.b)
            predictions = np.append(predictions,prediction)
    return predictions

In [14]:
alienpredic = load_predict('./data/validation/alien')

In [15]:
predatorpredic = load_predict('./data/validation/predator')

In [16]:
alienpredic

array([-1., -1., -1.,  1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1.,  1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        1., -1., -1., -1., -1., -1., -1., -1., -1.])

In [17]:
predatorpredic

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1.])

In [18]:
fp = 95
tp =5
tn = 100
fn = 0
p=100
n=100
akurasi = (tp+tn)/(p+n)*100
akurasi

52.5

In [19]:
error = 100 - akurasi
error

47.5

In [20]:
precision = tp/(tp+fp)*100
precision

5.0

In [21]:
recall = tp/(tp+fn)*100
recall

100.0

In [22]:
f1 = 2*precision*recall/(precision+recall)
f1

9.523809523809524